In [2]:
import tushare as ts
ts.set_token('4f669e3681eea3d45a6b30a64647bda7eef9e35c93d834ae4114b413')#token can be registered at https://tushare.pro/#
pro = ts.pro_api()
import baostock as bs
import pandas as pd
import akshare as ak
import numpy as np
import datetime
import time
import talib as ta
from talib import abstract
import datetime
import numpy as np

In [3]:
import os
import sys
import csv
import pandas as pd

In [4]:
lg = bs.login()

login success!


In [5]:
def bao_2_df(bsquery):
    rs = bsquery
    rlist = []
    result = []
    while ((rs.error_code == '0') & rs.next()):
        rlist.append(rs.get_row_data())
    result = pd.DataFrame(rlist, columns=rs.fields)
    print("done!")
    return result

In [6]:
#Getting used to api#

In [6]:
ts.get_realtime_quotes('000546')

,name,open,pre_close,price,high,low,bid,ask,volume,amount,...,a2_p,a3_v,a3_p,a4_v,a4_p,a5_v,a5_p,date,time,code
0,金圆股份,10.830,10.790,10.790,10.950,10.740,10.790,10.800,1929812,20948760.080,...,10.810,10,10.820,472,10.850,361,10.860,2020-06-10,10:02:03,000546


In [7]:
df=ts.get_hs300s().sort_values(by = ["weight"],ascending = False)
print(df.head(30))

          date    code   name  weight
131 2020-05-29  601318   中国平安    5.87
65  2020-05-29  600519   贵州茅台    5.26
16  2020-05-29  600036   招商银行    2.67
43  2020-05-29  600276   恒瑞医药    2.24
221 2020-05-29  000858  五 粮 液    2.20
..         ...     ...    ...     ...
256 2020-05-29  002415   海康威视    0.79
163 2020-05-29  601888   中国国旅    0.76
299 2020-05-29  300498   温氏股份    0.75
89  2020-05-29  600837   海通证券    0.72
210 2020-05-29  000661   长春高新    0.71

[30 rows x 4 columns]


In [8]:
####
# 已知的量化策略
# 
## svm量化策略 //胜率较低
# 周频
# 训练集：过去一年
# 观测15个交易日
# 1.收盘价/均值 2.现量/均量 3.最高价/均价 4.最低价/均价 5.现量 6.区间收益率 7.区间标准差
# 
# 若没有仓位则在每个星期一的时候输入标的股票近15个交易日的特征变量进行预测,并在预测结果为上涨的时候购买标的.
# 若已经持有仓位则在盈利大于10%的时候止盈,在星期五损失大于2%的时候止损.
#
## 指数增强 //胜率较高
# 略以0.8为初始权重跟踪指数标的沪深300中权重大于0.35%的成份股.
# 个股所占的百分比为(0.8*成份股权重)*100%.然后根据个股是否:
#   1.连续上涨5天 2.连续下跌5天
#   来判定个股是否为强势股/弱势股,并对其把权重由0.8调至1.0或0.6
#
## 行业滚动 //胜率0.4，考虑到不是分类而是选取问题，可以视为平均
# 月频
#  20个交易日的收益率并选取了收益率最高的指数的成份股获取并获取了他们的市值数据
#  随后把仓位调整至市值最大的5只股票上
#
## 集合竞价 //胜率0.5
# 本策略通过获取SHSE.000300沪深300的成份股数据并统计其30天内
# 开盘价大于前收盘价的天数,并在该天数大于阈值10的时候加入股票池
# 随后对不在股票池的股票平仓并等权配置股票池的标的,每次交易间隔1个月
#
## 多因子选股 //胜率0.65
# 基于Famma-French模型的策略
# 
# 订单流
# 主动性买盘 = 3 主动性卖盘
####

In [9]:
#Frequent pattern Mining#

In [10]:
# 2018-2019#
# 2015-2016#

In [8]:
def codetransform(plaincode,api):
    switch = {"0": "sz",
              "2": "sz",
              "3": "sz",
              "5": "sh",
              "6": "sh",
              "7": "sh",
              "9": "sh"
             }
    market = switch.get(plaincode[0:1])
    if api == "ts":
        return plaincode
    elif api == "ak":
        return str(market+plaincode)
    elif api == "bs":
        return str(market+"."+plaincode)
    elif api == "ts_pro":
        return str(plaincode+"."+market.upper())
    else:
        print("CodeTransform Error: api not recognized")
        return plaincode

In [9]:
def gen_dataset(startyr,endyr):
    start = str(startyr)+"-01-01"
    end = str(endyr)+"-12-31"
    code = []
    dfs = []
    for cod in ts.get_hs300s()["code"].values:
        bs.login()
        time.sleep(0.5)
        code.append(cod)
        df_d = bao_2_df(bs.query_history_k_data_plus(code = codetransform(cod,"bs"),start_date=start,end_date=end,fields="code,date,open,high,low,close,volume",frequency="d"))
        bs.logout()
        rs = []
        for tp in [4,6,9,12,26]:
            params = {
                "timeperiod" : tp
            }
            names = ["ema","dema"]
            rs = gen_indicators(names,df_d,params)
            for i in range(0,len(names)):
                if len(rs[i].keys())>1:
                    for j in range(0,len(rs[i].keys())):
                        key = rlist(s[i].keys())[j]
                        df_d[str(key+"_"+str(tp))] = rs[i][key]
                else:
                    df_d[str(names[i]+"_"+str(tp))] = rs[i]['real']
        for [fp,sp,sig] in [[12,26,9],[9,18,6],[15,30,10]]:#9,6,10
            params = {
                "fastperiod":fp,
                "slowperiod":sp,
                "signalperiod":sig,
            }
            names = ["macd"]
            rs = gen_indicators(names,df_d,params)
            for i in range(0,len(names)):
                if len(rs[i].keys())>1:
                    for j in range(0,len(rs[i].keys())):
                        key = list(rs[i].keys())[j]
                        df_d[str(key+"_"+str(sig))] = rs[i][key]
                else:
                    df_d[str(names[i]+"_"+str(sig))] = rs[i]['real']
        params = {
            "acceleration":0.02,
            "maximum":0.2,
            "fastk_period":5, 
            "slowk_period":3, 
            "slowk_matype":0, 
            "slowd_period":3,
            "slowd_matype":0
        }
        names = ["sar","stoch"]
        rs = gen_indicators(names,df_d,params)
        for i in range(0,len(names)):
                if len(rs[i].keys())>1:
                    for j in range(0,len(rs[i].keys())):
                        key = list(rs[i].keys())[j]
                        df_d[str(key)] = rs[i][key]
                else:
                    df_d[str(names[i])] = rs[i]['real']
        
        path = "./"+str(str(startyr)+str(endyr))+"/"+str(cod)+".csv"
        df_d.to_csv(path)
        dfs.append(df_d)
    return dfs

In [10]:
def gen_orderflow(code,date):
    time.sleep(0.1)
    df = ak.stock_zh_a_tick_tx_js(code =codetransform(str(code),"ak"))
    pass

In [11]:
def gen_indicators(names,df,params):
    info = {
        "open":df["open"],
        "high":df["high"],
        "low":df["low"],
        "close":df["close"],
        "volume":df["volume"]
    }
    rs = []
    for n in names:
        func = abstract.Function(n)
        dic = {}
        for key in func.parameters.keys():
            if (key not in params.keys()):
                print("Error in TA-lib: Wrong Params!")
                print(key+" Missing")
                print("Full Params: "+ str(func.parameters.keys()))
            dic[key] = params[key]
        func.parameters = dic
        if len(func.output_names)>1:
            rs_sub = {}
            for r in range(0,len(func.run(info))):
                rs_sub[func.output_names[r]] = func.run(info)[r]
            rs.append(rs_sub)
        else:
            rs.append(dict({func.output_names[0]:func.run(info)}))
    return rs

In [12]:
dfs=gen_dataset(2018,2019)

login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
l

done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
done!
logout success!
login success!
d

#Data Collection is pretty much done. Now cleaning the data by dropping NaN values(first 50 days of each year)#
#And by adding#

In [17]:
#Read the csv file, which is not necessary#
file_path = "./20182019/"
dfs = []
for file in os.listdir(file_path):
    if file.endswith(".csv"):
        dfs.append(pd.read_csv(str(str(file_path)+str(file))))

In [18]:
#Project the 7day and 15day precentageGain to previous dates#
for df in dfs:
    dff = df.copy()
    dff.sort_values(by = "open")
    length = len(df)
    pctGain_5 = []
    pctGain_20 = []
    for i,row in dff.iterrows():
        if (i+20) < length:
            close = row["close"]
            close_20 = dff["close"][i+20]
            close_5 = dff["close"][i+5]
            pct_5 = (close_5 - close)/close
            pct_20 = (close_20 - close)/close
            pctGain_5.append(pct_5)
            pctGain_20.append(pct_20)
        else:
            pctGain_5.append(np.nan)
            pctGain_20.append(np.nan)
    df["Gain_5"] = pctGain_5
    df["Gain_20"] = pctGain_20
dfsc = []
for df in dfs:
    dfsc.append(df.dropna())

In [19]:
dfsc[20].describe()

,Unnamed: 0,open,high,low,close,volume,ema_4,dema_4,ema_6,dema_6,...,macdsignal_6,macdhist_6,macd_10,macdsignal_10,macdhist_10,sar,slowk,slowd,Gain_5,Gain_20
count,417.000000,417.000000,417.000000,417.000000,417.000000,4.170000e+02,417.000000,417.000000,417.000000,417.000000,...,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000,417.000000
mean,258.000000,47.807050,48.434556,47.159257,47.799880,4.997487e+07,47.778564,47.797797,47.765647,47.798506,...,0.071541,-0.002940,0.119006,0.123508,-0.004502,47.962274,50.598176,50.617286,0.003820,0.017776
std,120.521782,7.776292,7.887183,7.633986,7.771966,3.273426e+07,7.676368,7.777190,7.625146,7.786362,...,0.761824,0.253721,1.009940,0.944285,0.324405,8.096779,24.835607,22.973626,0.049584,0.087794
min,50.000000,35.720000,36.030000,35.350000,35.680000,0.000000e+00,35.805807,35.644987,35.886132,35.648550,...,-1.393463,-1.060300,-1.700210,-1.520845,-0.995949,35.350000,5.121622,8.585216,-0.139876,-0.156680
25%,154.000000,39.710000,40.430000,39.110000,39.900000,3.243840e+07,39.610955,39.696792,39.579818,39.591721,...,-0.378738,-0.145646,-0.496443,-0.484858,-0.213447,40.449800,29.658087,31.809524,-0.027825,-0.038595
50%,258.000000,47.100000,47.700000,46.540000,47.210000,4.343216e+07,46.972187,47.166602,46.918126,46.982137,...,-0.003417,-0.001948,-0.060166,-0.029330,-0.025108,47.800000,49.911348,50.140976,0.000875,0.004414
75%,362.000000,54.700000,55.390000,54.100000,54.800000,5.740578e+07,54.530542,54.874449,54.568748,54.638724,...,0.431812,0.137598,0.772265,0.728142,0.208091,55.077529,72.853659,72.105212,0.034329,0.060661
max,466.000000,64.460000,65.400000,63.500000,65.000000,3.745625e+08,63.470845,64.432860,63.048455,64.230159,...,3.122108,0.825510,3.851665,3.204626,1.174674,65.400000,96.801826,96.293956,0.209913,0.404798


In [20]:
rowsum = 0
for i in range(0,len(dfsc)):
    rowsum += len(dfsc[i])
rowsum

121287

In [21]:
dfsc[0].columns

Index(['Unnamed: 0', 'code', 'date', 'open', 'high', 'low', 'close', 'volume',
       'ema_4', 'dema_4', 'ema_6', 'dema_6', 'ema_9', 'dema_9', 'ema_12',
       'dema_12', 'ema_26', 'dema_26', 'macd_9', 'macdsignal_9', 'macdhist_9',
       'macd_6', 'macdsignal_6', 'macdhist_6', 'macd_10', 'macdsignal_10',
       'macdhist_10', 'sar', 'slowk', 'slowd', 'Gain_5', 'Gain_20'],
      dtype='object')

In [22]:
ema_s = ["ema_4","ema_6","ema_9","dema_4","dema_6","dema_9"]
ema_l = ["ema_12", "dema_12","ema_26","dema_26"]
#macd, macdsignal, macdhist 9, 6 and 10
#sar
#slowk, slowd
signals = {
    "ema":[[[],[],[],[]],
            [[],[],[],[]],
            [[],[],[],[]],
            [[],[],[],[]],
            [[],[],[],[]],
            [[],[],[],[]]],
    "macd9":[],
    "macd6":[],
    "macd10":[],
    "sar":[],
    "stoch":[]
    }
G_5 = []
L_5 = []
G_20 = []
L_20 = []
for df in dfsc:
    df = df.reset_index()
    for i,row in df.iterrows():
        #ema#
        for s in range(0,len(ema_s)):
            for l in range(0,len(ema_l)):
                lastday = 0
                if (i-1) >= 0:
                    lastday = df[ema_s[s]][i-1] - df[ema_s[l]][i-1]
                if ((row[ema_s[s]] - row[ema_l[l]])>0) and (lastday<=0):
                    signals["ema"][s][l].append(1)
                else:
                    signals["ema"][s][l].append(0)
        #macd 6,9,10#
        for k in [6,9,10]:
            diff = "macd_"+str(k)
            dea = "macdsignal_"+str(k)
            macdsig = "macd"+str(k)
            lastday = 0
            if (i-1) >= 0:
                lastday = df[diff][i-1] - df[dea][i-1]
            if ((row[diff] - row[dea])>0) and ((lastday)<=0):
                signals[macdsig].append(1)
            else:
                signals[macdsig].append(0)
        #sar#
        if((row["close"]-row["sar"])>0):
            signals["sar"].append(1)
        else:
            signals["sar"].append(0)
        
        #stoch#
        if((row["slowk"] < 80 and row["slowd"] < 80)):
            signals["stoch"].append(1)
        else:
            signals["stoch"].append(0)
        
        if (row["Gain_5"] > 0.08):
            G_5.append(1)
            L_5.append(0)
        else:
            G_5.append(0)
            if (row["Gain_5"] <= -0.08):
                L_5.append(1)
            else:
                L_5.append(0)
        
        if (row["Gain_20"] > 0.2):
            G_20.append(1)
            L_20.append(0)
        else:
            G_20.append(0)
            if (row["Gain_20"]<=-0.2):
                L_20.append(1)
            else:
                L_20.append(0)

df_FP = pd.DataFrame()
for i in range(0,len(signals["ema"])):
    for j in range(0,len(signals["ema"][i])):
        string = str("ema"+"("+str(i)+";"+str(j)+")")
        df_FP[string] = signals["ema"][i][j]
for key in signals.keys():
    if key != "ema":
        df_FP[key] = signals[key]
df_FP["5dayGain"] = G_5
df_FP["20dayGain"] = G_20
df_FP["20dayLoss"] = L_20
df_FP["5dayLoss"] = L_5

In [23]:
print(len(df_FP))
df_FP.to_csv("./FP.csv")

121287
